# Persistent Cache

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

## Evaluation der Implementierung
Die Klasse `Cache` implementiert eine Transpositionstabelle die persistiert werden kann. Um zu ermitteln wie der Cache implementiert werden soll, wurde ein Geschwindigkeitsvergleich erstellt. Dieser ist im Noteboot `nmm-rote-performance-testing` zu finden. Das Ergebnis dieses Vergleichs hat ergeben, dass ein Python-Dictionary sich gut als Cache eignet.


## Cache
Die Klasse `Cache` implementiert eine Transpositionstabelle die persistiert werden kann.
Der Konstruktor erhält folgenden Parameter: 
- `max_size` gibt an, wie viele Einträge sich maximal im Cache befinden dürfen, bevor Einträge mit dem geringsten Rekursionslimit entfernt werden.
- `path` gibt an, aus welcher Datei der Cache wiederhergestellt werden soll. Ist dieser nicht gesetzt wird ein neuer leerer Cache initiiert.

In [ ]:
class Cache:
    def __init__(self, max_size = 1_000_000, path: str = None):
        self.max_size = max_size
        self.cache = {}
        if path:
            self.load(path)

Für Entwicklungszwecke wird eine Stringdarstellung für die Klasse `Cache` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
def __repr__(self: Cache) -> str:
    return f"Cache(size={len(self.cache)}, max_size={self.max_size})"

Cache.__repr__ = __repr__
del __repr__

Damit die Zustände und die Werte direkt als Bytes gespeichert und wieder ausgelesen werden können, ist das Paket `struct` nötig. Das Paket `tqdm` ermöglicht eine simple Fortschrittsanzeige.

In [ ]:
import struct
from tqdm.notebook import tqdm

Die Methode `convert_state_to_bytes` konvertiert einen Zustand in ein Byte-Array der Länge 7. Der Zustand wird dabei immer auf den weißen Spieler normiert. Durch die Normierung müssen die Zustände nur noch für einen Spieler gespeichert werden, worduch für die gleiche Menge an effektiv verwendbaren Zuständen nur noch die Hälfte des Speichers benötigt wird.
Dabei werden folgende Argumente erwartet:
- `state` $\in States$;
- `player` $\in Player$.

In [ ]:
def convert_state_to_bytes(state, player):
    players = ['w','b']
    if player == 'b':
        players = ['b','w']
        state = (state[0][::-1],state[1])
    byte_data = ((state[0][0] << 4) | state[0][1]).to_bytes(1,'big')
    for player in players:
        for ring in range(2,-1,-1):
            ring_byte = 0
            for cell in range(7,-1,-1):
                ring_byte <<= 1
                if state[1][ring][cell] is player:
                    ring_byte |= 1
            byte_data += ring_byte.to_bytes(1,'big')
    return byte_data

Die Methode `write` schreibt einen Zustand in die Transpositionstabelle. Dabei wird der Zustand auf den weißen Spieler normiert. Es werden folgende Argumente erwartet:
- `state` $\in States$;
- `player` $\in Player$;
- `limit` $\in \mathbb{N}_0$;
- `value` $\in \mathopen[-1.0,1.0\mathclose]$;
- `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
- `beta` $\in \mathopen[-1.0,1.0\mathclose]$.

In [ ]:
def write(self: Cache, state, player: str, limit: int, value: float, alpha: float, beta: float) -> None:
    state = convert_state_to_bytes(state, player)
    key = state + limit.to_bytes(1,'big')
    if player == 'b':
        value, alpha, beta = -value, -beta, -alpha
    value = struct.pack("d", value) + struct.pack("d",alpha) + struct.pack("d",beta)
    self.cache[key] = value

Cache.write = write
del write

Die Methode `read` liest einen vorher gespeicherten Zustand aus der Transpositionstabelle aus. Falls der Zustand nicht vorhanden ist wird `None` zurück gegeben. Dabei wird die Normierung auf den weißen Spieler rückgängig gemacht. Folgende Argumente werden erwartet:
- `state` $\in States$;
- `player` $\in Player$;
- `limit` $\in \mathbb{N}_0$.

Zurückgegeben wird ein Tripel bestehend aus:
1. `value` $\in \mathopen[-1.0,1.0\mathclose]$;
2. `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
3. `beta` $\in \mathopen[-1.0,1.0\mathclose]$.

In [ ]:
def read(self: Cache, state, player: str, limit: int) -> (float,float,float):
    state = convert_state_to_bytes(state, player)
    key = state + limit.to_bytes(1,'big')
    result = self.cache.get(key)
    if not result:
        return None
    value = struct.unpack("d", result[:8])[0]
    alpha = struct.unpack("d", result[8:16])[0]
    beta = struct.unpack("d", result[16:24])[0]
    if player == 'b':
        value, alpha, beta = -value, -beta, -alpha
    return (value, alpha, beta)

Cache.read = read
del read

Die Methode `clean` prüft ob der Cache seine maximale Größe überschritten hat. Ist dies der Fall werden Einträge beginnend mit dem Rekursionslimit $limit = 0$ aus dem Cache entfernt. Solange die Transpositionstabelle weiterhin ihre maximale Größe überschreitet, wird das minimale Rekursionslimit erhöht und die unterschreitenden Einträge gelöscht.

In [ ]:
def clean(self: Cache):
    min_limit = 0
    
    while len(self.cache) > self.max_size:
        min_limit += 1
        
        pre_len = len(self.cache)
        self.cache = {
            key: value
            for key, value in self.cache.items()
            if key[7] > min_limit
        }
        
        print(f"Increased min_limit to {min_limit} and deleted {pre_len - len(self.cache)} entries. " + \
              f"Cache is now {len(self.cache)} entries big.")
    
    return min_limit != 0

Cache.clean = clean
del clean

Die Methode `save` persistiert den Cache auf dem Dateisystem des Computers. Dafür wird folgendes Argument erwartet:
- `path` beschreibt den Pfad zur Cache-Datei im Dateisystem.

In [ ]:
def save(self: Cache, path: str):
    with open(path, "wb") as file:
        for key,value in tqdm(self.cache.items()):
            file.write(key)
            file.write(value)

Cache.save = save
del save

Die Methode `load` lädt einen vorhandenen Cache basierend auf einer Cache-Datei, die sich auf dem Dateisystem des Computers befindet. Dafür wird folgendes Argument erwartet:
- `path` beschreibt den Pfad zur Cache-Datei im Dateisystem.

In [ ]:
def load(self: Cache, path: str):
    if not os.path.isfile(path):
        print(f'Failed to load cache from file {path}!')
        return
    with open(path, "rb") as file:
        while True:
            key = file.read(8)
            value = file.read(24)
            if not key or not value:
                break
            self.cache[key] = value
    print(f'Successfully loaded cache from file {path}!')
Cache.load = load
del load

In [ ]:
cache = Cache()

state1 = s0 = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))

state2  = ((7, 6), (
    (' ', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', 'b'),
    (' ', ' ', ' ', ' ', ' ', 'w', ' ', ' ')
))

state2_2  = ((6, 7), (
    (' ', 'b', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'w', ' ', ' ', ' ', 'w'),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))

cache.write(state = state1, player = 'w', limit = 0, value = 0.3, alpha = 0.7, beta = -0.4)
cache.write(state = state2, player = 'b', limit = 2, value = 0.6, alpha = 0.8, beta = -0.6)

cache.save(path = 'test.cache')

del cache

cache2 = Cache(path = 'test.cache')

print(cache2.read(state = state1, player = 'w', limit = 0))
print(cache2.read(state = state2, player = 'b', limit = 2))
print(cache2.read(state = state2_2, player = 'w', limit = 2))
